# (Kaggle) Titanic: Machine Learning from Disaster Chap4

## 시작하기 전에

* 타이타닉 해상사고 데이터 분석을 진행하기 앞서 사용했던 파이썬 패키지 정리
    1. pip install numpy
    2. pip install scipy
    3. pip install pandas
    4. pip install scikit-learn
    5. pip install matplotlib
    6. pip install seaborn
    7. pip install jupyter

## 머신러닝 알고리즘 최적화(하이퍼파라미터 튜닝, 다른 알고리즘 적용)

이전까지 해서 데이터 전처리 및 머신러닝 알고리즘 적용까지 실습을 하였습니다. 추가적으로 다른 알고리즘도 적용해 보고 필요에 따라서 **hyperparameter tuning**을 진행해 보겠습니다.

In [8]:
# pandas 패키지를 사용
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# 트리 시각화 모듈(graphviz)정의
import graphviz
# scikit-learn모듈에서 Decision Tree를 시각화 할 수 있는 export_graphviz를 불러옴
from sklearn.tree import export_graphviz

# 머신러닝을 사용하기 위한 패키지 import
# 추가적으로 다른 패키지(알고리즘)도 추가
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

# 블로그 업데이트용 jupyter notebook 크기 수정하는 부분(선택사항)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 97% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# 수정한 데이터 불러오기
train = pd.read_csv("data/train_modify_final.csv", index_col="PassengerId")
test = pd.read_csv("data/test_modify_final.csv", index_col="PassengerId")

In [4]:
# feature들에 해당하는 명칭을 리스트에 저장합니다.
feature_names = ["Pclass", "Sex_encode", "Fare_fill", "Embarked_C", "Embarked_S", "Embarked_Q",
                 "is_Child", "is_Master", "Single", "Big", "Nuclear"]

feature_names

['Pclass',
 'Sex_encode',
 'Fare_fill',
 'Embarked_C',
 'Embarked_S',
 'Embarked_Q',
 'is_Child',
 'is_Master',
 'Single',
 'Big',
 'Nuclear']

In [5]:
# 생존 여부(Survived)를 label로 정의
label_name = "Survived"
label_name

'Survived'

In [6]:
# 학습을 시킬 feature 데이터만 추출합니다
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(891, 11)


,Pclass,Sex_encode,Fare_fill,Embarked_C,Embarked_S,Embarked_Q,is_Child,is_Master,Single,Big,Nuclear
PassengerId,,,,,,,,,,,
1,3,0.0,7.2500,False,True,False,False,False,False,False,True
2,1,1.0,71.2833,True,False,False,False,False,False,False,True
3,3,1.0,7.9250,False,True,False,False,False,True,False,False
4,1,1.0,53.1000,False,True,False,False,False,False,False,True
5,3,0.0,8.0500,False,True,False,False,False,True,False,False


In [7]:
# test 데이터도 같은 방식으로 진행합니다.
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(418, 11)


,Pclass,Sex_encode,Fare_fill,Embarked_C,Embarked_S,Embarked_Q,is_Child,is_Master,Single,Big,Nuclear
PassengerId,,,,,,,,,,,
892,3,0.0,7.8292,False,False,True,False,False,True,False,False
893,3,1.0,7.0000,False,True,False,False,False,False,False,True
894,2,0.0,9.6875,False,False,True,False,False,True,False,False
895,3,0.0,8.6625,False,True,False,False,False,True,False,False
896,3,1.0,12.2875,False,True,False,False,False,False,False,True


In [12]:
# y_train 생성
y_train = train[label_name]

print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### Anoter algorithm application

이번에는 **Decision Tree**가 아닌 **다른 머신러닝 알고리즘들**을 사용하여 score를 내보겠습니다. 

#### 기존에 적용했던 Decision Tree 모델(일부 파라미터 값 적용)

In [20]:
# 기존에 적용했던 Decision Tree 모델
decision_old = DecisionTreeClassifier(max_depth=8, random_state=0)
decision_old.fit(X_train, y_train)
predictions_DT_old = decision_old.predict(X_test)
print(predictions.shape)

# score 매소드를 이용하면 어느정도 정확도가 나올지 어림잡아 예측할 수 있다.
acc_decision_tree_old = round(decision_old.score(X_train, y_train) * 100, 2)
acc_decision_tree_old

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

(418,)


88.66

결과는 **88.66**이 나오는데 실제 제출했을 때 정확도(**0.81339**)와는 차이가 있지만 이 수치가 큼에 따라 정확도가 올라갈 수도 있다는 것을 예측해 볼 수 있습니다.

#### Decision Tree default parameter 모델(일부 파라미터 값 적용)

In [21]:
# DecisionTreeClassifier에서 파라미터를 모두 제거하여 Default 값으로 테스트 해 봅니다.
decision_new = DecisionTreeClassifier()
decision_new.fit(X_train, y_train)
predictions_DT_new = decision_new.predict(X_test)
print(predictions.shape)

# score 매소드를 이용하면 어느정도 정확도가 나올지 어림잡아 예측할 수 있다.
acc_decision_tree_new = round(decision_new.score(X_train, y_train) * 100, 2)
acc_decision_tree_new

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

(418,)


92.37

해당 파라미터들을 모두 **default**값으로 적용시켰을 시, 92.37로 스코어가 더 상승하는 것을 확인했습니다. 해당 예측값을 파일 포멧에 적용하여 점수가 얼마나 오르는지 확인해 보겠습니다.

In [19]:
# 제출 파일 포멧 불러오기
submission = pd.read_csv("data/gender_submission.csv", index_col="PassengerId")

# 제대로 나오나 확인
print(submission.shape)
submission.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [22]:
# 제출 포멧(submission)의 생존 여부(Survived) 컬럼에 우리의 예측값(predictions)를 집어넣습니다.
# 두 데이터 모두 길이가 418개로 동일하기 때문에, 등호(=)를 통해 쉽게 예측값을 넣을 수 있습니다.
submission["Survived"] = predictions_DT_new

# submission 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(submission.shape)
submission.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [25]:
# 파일 형태로 다시 저장합니다.
submission.to_csv("data/DT_new_prediction.csv")

여기서 알 수 있는 사실이 하나 있습니다. score 메소드를 통해 내는 것은 training한 데이터를 가지고 자체 테스트를 하는 것이기 때문에, 설사 점수가 올라가더라도 **Overfitting**(너무 주어진 데이터에 맞게만 학습)으로 인해 실제 제출 결과에서 하락하는 결과를 얻었습니다.(**제출결과 : 0.80861**) DT 특성상 가지가 너무 많이 쳐지게 되면 학습한 데이터에 **너무 과도하게 맞기** 쉽기 때문에 Default 파라미터의 예측 정확도가 더 떨어지지 않았나 생각이 듭니다.

#### Logistic Regression(선형회귀) 적용

In [27]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)
prediction_lg = logistic.predict(X_test)
acc_logistic = round(logistic.score(X_train, y_train) * 100, 2)
acc_logistic

c:\users\smart\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

82.94

오히려 score가 이전에 적용했던 **Decision Tree**보다 많이 낮게 나옵니다. 실제 결과는 어떨지 테스트 해 보겠습니다.

In [28]:
submission["Survived"] = prediction_lg

# submission 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(submission.shape)
submission.head()

submission.to_csv("data/LG_prediction.csv")

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


실제 제출 결과도 **DT**의 결과보다 낮게 나왔습니다.(**0.78468**)

#### SVM(Support Vector Machine) 적용

In [29]:
svc = SVC()
svc.fit(X_train, y_train)
prediction_svc = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc

c:\users\smart\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

82.15

In [30]:
submission["Survived"] = prediction_svc

# submission 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(submission.shape)
submission.head()

submission.to_csv("data/SVC_prediction.csv")

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


최종 스코어 : (**0.72727**)

#### Random Forest 적용

In [34]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
prediction_rf = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

0.9236812570145904

92.37

In [35]:
submission["Survived"] = prediction_rf

# submission 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(submission.shape)
submission.head()

submission.to_csv("data/RF_prediction.csv")

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


최종 스코어 : (**0.78468**)

### 알 수 있었던 점

각각 다른 **ML Algorithm**을 적용해 보면서, 실제 학습 데이터 score와 예측 정확도가 일치하지 않는 것과, 절대적으로 우위적인 **ML Algorithm**이 없다는 것을 알게 되었습니다. 데이터의 양과, 경향성 그리고 어떤 방식으로 데이터를 가공하냐에 따라 맞는 알고리즘이 존재한다는 것을 알게되었습니다.

### 출처

* Jupter notebook 블로그 paste 관련 전처리 :  https://versusall.tistory.com/entry/1-Tensor-Flow-%EC%8B%9C%EC%9E%91%ED%95%98%EA%B8%B0-Core-Tutorial?category=749169
* seaborn 데이터 시각화에 대한 정리 글 : https://brunch.co.kr/@jjason68/12
* seaborn 데이터 시각화에 대한 정리 글2 : https://pinkwink.kr/984
* DSSchool 기초과정에서 배운 내용을 토대로 재구성 및 복습
* Kaggle kernel "Titanic Data Science Solutions": https://www.kaggle.com/startupsci/titanic-data-science-solutions